In [7]:
# using NLopt
using Optim
using StatsBase
using Combinatorics

include("jl/omega.jl")
include("jl/HSBM.jl")
include("jl/hypergraph_louvain.jl")
include("jl/objectives.jl")
include("jl/inference.jl");

Ok, so I think the way this works is that `myfunc` is going to supply the update in the gradient by modifying it in place. This seems very smart -- each function call updates the gradient. So, what we would need to do evaluate the relevant sums and cut terms, then follow this pattern. 

In [31]:
# parameters

n = 100
Z = rand(1:10, n)
ϑ = dropdims(ones(1,n) + rand(1,n), dims = 1)

# defining group intensity function Ω
μ = mean(ϑ)

ω(p,α) = maximum(1.0*p)^α[1] / (sum((1.0*p).^α[1])*α[2])

α0 = [100.0, 1e6]

kmax = 4

Ω = buildΩ(ω, α0, kmax)

Ω (generic function with 1 method)

In [32]:
H = sampleSBM(Z, ϑ, Ω; α=α0, kmax=kmax, kmin = 1)

hypergraph
  N: Array{Int64}((100,)) [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
  E: Dict{Int64,Dict}
  D: Array{Int64}((100,)) [13, 9, 5, 5, 17, 4, 12, 13, 13, 9  …  11, 9, 8, 10, 13, 9, 10, 6, 6, 9]


In [33]:
println("The log-likelihood of the true partition is $(round(logLikelihood(H, Z, Ω, ϑ;α=α0),digits=3)).")
println("The log-likelihood of the singleton partition is $(round(logLikelihood(H, collect(1:n), Ω, ϑ;α=α0),digits=3)).")

The log-likelihood of the true partition is -2797.908.
The log-likelihood of the singleton partition is -2917.276.


In [34]:
α̂ = α0
Ẑ = Z
for i =1:5
    Z_prev = Ẑ
    Ẑ = HyperLouvain(H,kmax,Ω;α=α̂)
#     println(Z_prev==Ẑ)
    
    obj = parameterEstimateObjective(H, Ẑ, Ω)
    
    res = optimize(obj, 0, [10.0, 100.0],
            NelderMead(),
            Optim.Options(f_tol = 1e-10,
                iterations = 1000,
                show_trace=false))
    println(-Optim.minimum(res))
    println(Optim.minimizer(res) )
    α_prev = α̂
    α̂ = Optim.minimizer(res) 
end


Louvain Iteration 1
No nodes moved clusters
-9887.95181039062
[-90.31852465254224, 8.813185221919677e7]

Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
-9345.85961526854
[-11.679200433677124, 6.946377981150708e7]

Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
Louvain Iteration 4
Louvain Iteration 5
Louvain Iteration 6
-137274.67918423985
[-22.660755823091847, 1.1520297666166965e6]

Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
Louvain Iteration 4
Louvain Iteration 5
Louvain Iteration 6
Louvain Iteration 7
Louvain Iteration 8
Louvain Iteration 9
-36151.68533162755
[-51.613618135986655, 4.801264229159951e6]

Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
Louvain Iteration 4
Louvain Iteration 5
-68549.78855181887
[-292.0250334059525, 3.1677741590961246e6]
